In [34]:
import math
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import BasicAer, execute

In [35]:
# decimal_to_binary_two_compl function returns the 2's complement binary representation of an integer 
# in QUBIT_SIZE many bits
def decimal_to_binary_two_compl(num):
    if (num & (1 << (QUBIT_SIZE - 1))) != 0: # checking if sign bit is set i.e. if integer is negative
        numbin = bin(num & (2**QUBIT_SIZE - 1)).replace("0b","")
    else:
        numbin = bin(num).replace("0b","").zfill(QUBIT_SIZE) 
    
    return numbin

# returns the quantum circuit constructed for comparing the two given integers
def construct_circuit(num1, num2):
    
    num1bin = decimal_to_binary_two_compl(num1)
    num2bin = decimal_to_binary_two_compl(num2)

    # binary representation of two integers are reversed since qiskit uses little-endian bit ordering
    num1bin = num1bin[::-1] 
    num2bin = num2bin[::-1]

    int1 = QuantumRegister(QUBIT_SIZE, 'first_int') # first integer register
    int2 = QuantumRegister(QUBIT_SIZE, 'second_int') # second integer register
    int2_copy = QuantumRegister(QUBIT_SIZE, 'second_int_copy' ) # copy of second integer
    diff1bit = QuantumRegister(QUBIT_SIZE, 'first_diff_bit') # if ith bit is the first different bit 
                                                                # among two integers, then only ith bit is set
    int1_AND_diff1bit = QuantumRegister(QUBIT_SIZE, 'first_int_AND_first_diff_bit') # stores bitwise AND between 
                                                                                    # int1 and diff1bit registers 
    int2_AND_diff1bit = QuantumRegister(QUBIT_SIZE, 'second_int_AND_first_diff_bit') # stores bitwise AND between 
                                                                                    # int2_copy and diff1bit registers 
    OR_int1bits = QuantumRegister(1, 'afterAND_OR_first_int') # stores OR of the bits of int1_AND_diff1bit register
    OR_int2bits = QuantumRegister(1, 'afterAND_OR_second_int') # stores OR of the bits of int2_AND_diff1bit register
    meas_OR_int1bits = ClassicalRegister(1, 'measure_OR_first_int') # stores classical value of OR_int1bits register
    meas_OR_int2bits = ClassicalRegister(1, 'measure_OR_second_int') # stores classical value of OR_int2bits register

    comparator_circuit = QuantumCircuit(int1, int2, int2_copy, diff1bit, int1_AND_diff1bit, int2_AND_diff1bit,
                                        OR_int1bits, OR_int2bits, meas_OR_int1bits, meas_OR_int2bits)

    # circuits for encoding the two integers, except the sign bit, insert X if the corresponding bit in the binary 
    # representation is 1 
                                                                                                
    # encodes first integer 
    for i in range(QUBIT_SIZE-1):
        if num1bin[i] == '1':
            comparator_circuit.x(int1[i])

    # encodes second integer 
    for i in range(QUBIT_SIZE-1):
        if num2bin[i] == '1':
            comparator_circuit.x(int2[i])
    
    # for ease of comparison, flip the sign bit of both integers i.e. positive integer now has sign bit 1 and 
    # negative integer now has sign bit 0 
    if num1bin[QUBIT_SIZE-1] == '0':
        comparator_circuit.x(int1[QUBIT_SIZE-1])
        
    if num2bin[QUBIT_SIZE-1] == '0':
        comparator_circuit.x(int2[QUBIT_SIZE-1])

    control_list = [] # list of control qubits for multi-controlled toffoli (MCT) gate

    # fanout operation for copying classical bit values of second integer to the register int2_copy
    for i in range(QUBIT_SIZE):
        comparator_circuit.cnot(int2[i], int2_copy[i]) 

    # circuit for determining the first different bit position among two integers
    for i in range(QUBIT_SIZE-1, -1, -1):
        comparator_circuit.cnot(int1[i], int2[i])
        if i < QUBIT_SIZE - 1:
            comparator_circuit.x(int2[i + 1])
        control_list.append(int2[i])
        comparator_circuit.mct(control_list, diff1bit[i])

    # Bitwise AND between int1 and diff1bit registers using regular two-control Toffoli gates
    for i in range(QUBIT_SIZE):
        comparator_circuit.ccx(int1[i], diff1bit[i], int1_AND_diff1bit[i])

    # Bitwise AND between int2_copy and diff1bit registers using regular two-control Toffoli gates
    for i in range(QUBIT_SIZE):
        comparator_circuit.ccx(int2_copy[i], diff1bit[i], int2_AND_diff1bit[i])

    # Use CNOTs to compute OR among the bits of int1_AND_diff1bit, store it in OR_int1bits register
    # Use CNOTs to compute OR among the bits of int2_AND_diff1bit, store it in OR_int2bits register
    # if the two numbers are not equal, one of the two registers (either OR_int1bits or OR_int2bits) 
    # will have value 1 and the other will have value 0, if OR_int1bits is 1, then num1 is larger and vice-versa
    # if the two numbers are equal, both registers will have value zero
    for i in range(QUBIT_SIZE):
        comparator_circuit.cx(int1_AND_diff1bit[i], OR_int1bits[0])
        comparator_circuit.cx(int2_AND_diff1bit[i], OR_int2bits[0])
    
    # measure OR_int1bits and OR_int2bits
    comparator_circuit.measure(OR_int1bits, meas_OR_int1bits)
    comparator_circuit.measure(OR_int2bits, meas_OR_int2bits)

    return comparator_circuit

In [36]:
# returns the larger number
def find_the_largest_number(num1, num2):
    global equal_flag
    comparator_circuit = construct_circuit(num1,num2)
    comparator_circuit.draw()
    simulator = BasicAer.get_backend('qasm_simulator')
    result = execute(comparator_circuit, backend = simulator).result().get_counts()

    value = list(result.keys())[0]
    
    if value == '1 0': 
        return num2, comparator_circuit

    elif value == '0 1':
        return num1, comparator_circuit
    
    else:    
        equal_flag = 1
        return num1, comparator_circuit

In [37]:
QUBIT_SIZE = 3 # QUBIT_SIZE is set to 3 because QASM_SIMULATOR only supports maximum 24 qubits
equal_flag = 0 # flag is set if the two entered integers are equal

# the allowed range for num1 and num2 is [-2^(QUBIT_SIZE-1), 2^(QUBIT_SIZE-1)-1]
num1 = int(input("Enter the first integer"))
num2 = int(input("Enter the second integer"))

larger_num, comparator_circuit = find_the_largest_number(num1, num2)

comparator_circuit.draw()


Enter the first integer1
Enter the second integer1


┌───┐                                        »
                    first_int_0: ┤ X ├──────────────────────■─────────────────»
                                 └───┘                      │                 »
                    first_int_1: ─────────────────■─────────┼─────────────────»
                                 ┌───┐            │         │                 »
                    first_int_2: ┤ X ├────────────┼────■────┼─────────■───────»
                                 ├───┤            │    │  ┌─┴─┐       │       »
                   second_int_0: ┤ X ├───────■────┼────┼──┤ X ├───────┼───────»
                                 └───┘       │  ┌─┴─┐  │  └───┘       │       »
                   second_int_1: ───────■────┼──┤ X ├──┼──────────────┼────■──»
                                 ┌───┐  │    │  └───┘┌─┴─┐     ┌───┐  │    │  »
                   second_int_2: ┤ X ├──┼────┼────■──┤ X ├──■──┤ X ├──┼────■──»
                                 └───┘  │  ┌─┴─┐  │  └───┘  │  └───┘  │    │  »
              second_int_copy_0: ───────┼──┤ X ├──┼─────────┼─────────┼────┼──»
                                      ┌─┴─┐└───┘  │         │         │    │  »
              second_int_copy_1: ─────┤ X ├───────┼─────────┼─────────┼────┼──»
                                      └───┘     ┌─┴─┐       │         │    │  »
              second_int_copy_2: ───────────────┤ X ├───────┼─────────┼────┼──»
                                                └───┘       │         │    │  »
               first_diff_bit_0: ───────────────────────────┼─────────┼────┼──»
                                                            │         │  ┌─┴─┐»
               first_diff_bit_1: ───────────────────────────┼─────────┼──┤ X ├»
                                                          ┌─┴─┐       │  └───┘»
               first_diff_bit_2: ─────────────────────────┤ X ├───────■───────»
                                                          └───┘       │       »
 first_int_AND_first_diff_bit_0: ─────────────────────────────────────┼───────»
                                                                      │       »
 first_int_AND_first_diff_bit_1: ─────────────────────────────────────┼───────»
                                                                    ┌─┴─┐     »
 first_int_AND_first_diff_bit_2: ───────────────────────────────────┤ X ├─────»
                                                                    └───┘     »
second_int_AND_first_diff_bit_0: ─────────────────────────────────────────────»
                                                                              »
second_int_AND_first_diff_bit_1: ─────────────────────────────────────────────»
                                                                              »
second_int_AND_first_diff_bit_2: ─────────────────────────────────────────────»
                                                                              »
          afterAND_OR_first_int: ─────────────────────────────────────────────»
                                                                              »
         afterAND_OR_second_int: ─────────────────────────────────────────────»
                                                                              »
         measure_OR_first_int: 1/═════════════════════════════════════════════»
                                                                              »
        measure_OR_second_int: 1/═════════════════════════════════════════════»
                                                                              »
«                                                                              »
«                    first_int_0: ──────────────────────■──────────────────────»
«                                                       │                      »
«                    first_int_1: ───────■──────────────┼──────────────────────»
«                                        │              │                      »
«                    first_i

In [38]:
if equal_flag!=1:
    print("the larger number is ", larger_num)
else:
    print("two numbers are equal")



two numbers are equal
